In [1]:
# from datasets import load_dataset
import os, collections, json, random
import xml.etree.ElementTree as ET
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

In [3]:
# fm.py
def load_file(filepath, ):
    with open(filepath, encoding='utf-8') as f:
        a = []
        for i in f:
            a.append(i.strip(' \n'))
        return a


def load_dict_json(filepath, ):
    with open(filepath, encoding='utf-8') as f:
        a = json.load(f)
    return a


def save_file(filepath, lst):
    with open(filepath, 'w', encoding='utf-8') as f:
        for i in range(len(lst)):
            f.write(lst[i])
            if i != len(lst) - 1:
                f.write('\n')


def save_dict_json(filepath, dic):
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(json.dumps(dic, ensure_ascii=False))


def load_xml(filepath):
    tree = ET.ElementTree()
    tree = ET.parse(filepath)
    children = tree.findall('DOC')
    a = []
    for i in children:
        for j in i[0]:
            a.append(j.text)
    return a


def get_filelist(dir, Filelist):
    newDir = dir
    if os.path.isfile(dir):
        Filelist.append(dir)
        # 只返回文件名
        # Filelist.append(os.path.basename(dir))
    elif os.path.isdir(dir):
        for s in os.listdir(dir):
            # 若需要忽略文件夹
            # if s == "xxx":
            # continue
            newDir = os.path.join(dir, s)
            get_filelist(newDir, Filelist)
    return Filelist


def get_dir_list(dir_path):
    lst = []
    for s in os.listdir(dir_path):
        if os.path.isdir(os.path.join(dir_path, s)):
            lst.append(s)
    return lst

def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip('\n')
        vocab[token] = index
    return vocab

def save_vocabulary(vocab, vocab_path):
        """Save the tokenizer vocabulary to a directory or file."""
        index = 0
        with open(vocab_path, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(vocab.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    # logger.warning("Saving vocabulary to {}: vocabulary indices are not consecutive."
                    #                " Please check that the vocabulary is not corrupted!".format(vocab_file))
                    index = token_index
                writer.write(token + u'\n')
                index += 1
        return (vocab_path,)

# SST-5

In [3]:
sst_dataset = load_dataset("sst", "default")

Using the latest cached version of the module from /home/chwu/.cache/huggingface/modules/datasets_modules/datasets/sst/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff (last modified on Tue Nov 16 21:28:55 2021) since it couldn't be found locally at sst., or remotely on the Hugging Face Hub.
Reusing dataset sst (/home/chwu/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
sst_dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 8544
    })
    validation: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 1101
    })
    test: Dataset({
        features: ['sentence', 'label', 'tokens', 'tree'],
        num_rows: 2210
    })
})

In [5]:
# float label 转化
def label_trans(x):
    if x >= 0 and x <= 0.2:
        return 0
    if x > 0.2 and x <= 0.4:
        return 1
    if x > 0.4 and x <= 0.6:
        return 2
    if x > 0.6 and x <= 0.8:
        return 3
    if x > 0.8 and x <= 1:
        return 4

In [6]:
# 获取对应train, valid, test数据集
def get_data(all_dataset, splitname):
    sent = all_dataset[splitname]['sentence']
    label = all_dataset[splitname]['label']
    label = [label_trans(x) for x in label]
    return [str(l) + '\t' + s for l, s in zip(label, sent)]

In [11]:
train_data = get_data(sst_dataset, 'train')
valid_data = get_data(sst_dataset, 'validation')
test_data = get_data(sst_dataset, 'test')

In [13]:
output_dir = 'EmotionRecognition/SST-5'
save_file(os.path.join(output_dir, "data_train.txt"), train_data)
save_file(os.path.join(output_dir, "data_val.txt"), valid_data)
save_file(os.path.join(output_dir, "data_test.txt"), test_data)

# SMP-2020

In [24]:
data = pd.read_csv('EmotionRecognition/SMP-2020/train.tsv', header=None, sep='\t')

In [25]:
train, test = train_test_split(data, test_size=5000, shuffle=True, stratify=data[0])

In [26]:
train.to_csv('EmotionRecognition/SMP-2020/data_train.txt', sep='\t', index=None, header=None)
test.to_csv('EmotionRecognition/SMP-2020/data_test.txt', sep='\t', index=None, header=None)

# IECE-Emo

In [5]:
def load_dataset(input_file):
    df = pd.read_csv(input_file, header=None)
    # df = shuffle(df)
    X = df.iloc[:, [0]].values
    y = df.iloc[:, [2]].values
    return X, y

In [4]:
def split(full_data, shuffle=False, ratio = 0.1):
    n_total = len(full_data)
    offset = int(n_total * ratio)
    if n_total == 0 or offset < 1:
        return [], [], full_data
    if shuffle:
        random.shuffle(full_data)
    dev_set = full_data[:offset]
    test_set = full_data[offset:offset*2]
    train_set = full_data[offset*2:]
    return train_set, dev_set, test_set

In [ ]:
X,y = load_dataset('../dataset/Implicit_emo_dataset/all_data.csv')
print(set(y.reshape(-1).tolist()))
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
y = y.astype(str).astype(object)
X = X.astype(object)
y = y.reshape(-1,1)
x = (y+'\t'+X)
x = x.reshape(-1)
data = list(set(x.tolist()))
fm.save_file('../dataset/Implicit_emo_dataset/data.txt', data)

In [ ]:
train_set, dev_set, test_set = split(data, shuffle=True)
fm.save_file('../dataset/Implicit_emo_dataset/data_train.txt', train_set)
fm.save_file('../dataset/Implicit_emo_dataset/data_val.txt', dev_set)
fm.save_file('../dataset/Implicit_emo_dataset/data_test.txt', test_set)

# ISEAR

In [54]:
data = pd.read_csv('EmotionRecognition/ISEAR/isear.csv', sep='|')

In [55]:
X = data['SIT'].values
y = data['Field1'].values


In [56]:
set(y.reshape(-1).tolist())

{'anger', 'disgust', 'fear', 'guilt', 'joy', 'sadness', 'shame'}

In [57]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
y = y.astype(str).astype(object)
X = X.astype(object)
y = y.reshape(-1,1)
# x = (y+'\t'+X)
# x = x.reshape(-1)
# data = list(set(x.tolist()))

In [58]:
X = X.reshape(-1).tolist()
y = y.reshape(-1).tolist()

In [59]:
df_data = pd.DataFrame({'label':y, 'text':X})

In [60]:
train, test = train_test_split(df_data, test_size=0.2, random_state=7, stratify=df_data['label'])

In [61]:
val, test = train_test_split(test, test_size=0.5, random_state=7, stratify=test['label'])

In [63]:
train.to_csv('EmotionRecognition/ISEAR/data_train.txt', sep='\t', index=None, header=None)
val.to_csv('EmotionRecognition/ISEAR/data_val.txt', sep='\t', index=None, header=None)
test.to_csv('EmotionRecognition/ISEAR/data_test.txt', sep='\t', index=None, header=None)


# TEC

In [2]:
data = pd.read_csv('EmotionRecognition/TEC/Jan9-2012-tweets-clean.txt', sep='\t', header=None)

In [3]:
X = data[1].values
y = data[2].values

In [4]:
from collections import Counter
Counter(y)

Counter({':: surprise': 3849,
         ':: sadness': 3830,
         ':: joy': 8240,
         ':: disgust': 761,
         ':: fear': 2816,
         ':: anger': 1555})

In [16]:
set(y.reshape(-1).tolist())

{':: anger', ':: disgust', ':: fear', ':: joy', ':: sadness', ':: surprise'}

In [17]:
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
y = y.astype(str).astype(object)
X = X.astype(object)
X = X.reshape(-1).tolist()
y = y.reshape(-1).tolist()

In [20]:
df_data = pd.DataFrame({'label':y, 'text':X})
train, test = train_test_split(df_data, test_size=0.2, random_state=7, stratify=df_data['label'])
val, test = train_test_split(test, test_size=0.5, random_state=7, stratify=test['label'])

In [21]:
train.to_csv('EmotionRecognition/TEC/data_train.txt', sep='\t', index=None, header=None)
val.to_csv('EmotionRecognition/TEC/data_val.txt', sep='\t', index=None, header=None)
test.to_csv('EmotionRecognition/TEC/data_test.txt', sep='\t', index=None, header=None)